Notebook para RAG con Llama 3

In [1]:
!pip3 install ollama -q
!pip3 install langchain-community langchain-core



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [7]:
!pip3 install pypdf -q
!pip3 install chromadb


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [21]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

import ollama

In [20]:
loader = PyPDFLoader("independenciaCA.pdf")
paginas = loader.load()

In [22]:
print(len(paginas))

328


In [26]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 650, #Cada division de texto tendra 650 caracteres
    chunk_overlap = 60, #traslape de 40 caracteres
    length_function = len
)

In [13]:
docs = text_splitter.split_documents(paginas) #aplicar al recien text spliter definido

In [25]:
print(len(docs))
print(len(paginas))


855
328


In [15]:
embeddings = OllamaEmbeddings(model="llama3") #Seleccionamos Llama 3 para convertir nuestro documento a emmbeddings
vectorstore = Chroma.from_documents(documents=docs,embedding=embeddings)

In [16]:
def ollama_llm(question, context):
    formatted_prompt = f"Pregunta: {question}\n\nContexto (Responde solo sobre el contenido del texto entregado en el documento): {context}"
    response = ollama.chat(
        model="llama3",
        messages=[{'role':'user', 'content': formatted_prompt}],
        options={'temperature':0})
    return response['message']['content']

In [27]:
retriever = vectorstore.as_retriever()
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [28]:
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = combine_docs(retrieved_docs)
    return ollama_llm(question,formatted_context)

In [29]:
result = rag_chain("¿Que Guatemaltecos firmaron el acta de independencia de Guatemala")
print(result)

Según el texto, los Guatemaltecos que firmaron el acta de independencia de Guatemala fueron:

1. Mariano Antonio de Larrave (alcalde del Ayuntamiento y médico)
2. Antonio Rivera Cabezas (abogado y representante de la Diputación Provincial)
3. José Antonio Larrave y Velasco (abogado y regidor del Ayuntamiento)
4. Isidoro del Valle y Castriciones (regidor del Ayuntamiento, fecha de nacimiento y fallecimiento desconocida)
5. Mariano de Aycinena (síndico primero del Ayuntamiento y diputado por Quetzaltenango)

Estos personajes firmaron el acta de independencia de Guatemala en 1821.
